In [1]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
os.chdir(r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter3_SyntheticTCs\04_RESULTS\ncep')
scenario = 'compound'

da_list = []
for file in os.listdir():
    if ('zsmax' in file) & (file.endswith('.nc')):
        da = xr.open_dataarray(file, engine='netcdf4')
        ds = da.to_dataset(dim='scenario', promote_attrs=True)
        da_list.append(ds[scenario])
ds_zsmax = xr.concat(objs=da_list, dim='tc_id')

# Define return periods of interest
target_return_periods = [2, 5, 10, 25, 50, 100, 200, 500]
n_storms = len(ds_zsmax.tc_id)
print(n_storms)

In [4]:
# Step 1: Sort the entire dataset along the storm dimension
sorted_levels = np.sort(np.nan_to_num(ds_zsmax.values, nan=np.nan), axis=0)[::-1] # Sort from highest to lowest

In [43]:
# Step 2: Calculate the exceedance probabilities for all cells at once
probabilities = np.arange(1, n_storms + 1) / (n_storms + 1)

# Step 3: Calculate the return periods for all exceedance probabilities
return_periods = 1 / probabilities

# Step 4: Create an array to hold the return period water levels (shape: return_periods x lat x lon)
return_periods_data = np.zeros((len(target_return_periods), ds_zsmax.sizes['y'], ds_zsmax.sizes['x']))

# Step 5: Use broadcasting to find return period water levels for each target return period
for i, target_rp in enumerate(target_return_periods):
    # Find the index corresponding to the target return period for each cell
    idx = np.searchsorted(return_periods, target_rp)

    # Make sure we don't go out of bounds
    idx = np.clip(idx, 0, n_storms - 1)

    # Store the corresponding water levels for all cells at once
    return_periods_data[i, :, :] = sorted_levels[idx, :, :]

# Convert the result back to an xarray DataArray
return_periods_data_xr = xr.DataArray(return_periods_data,
                                      dims=["return_period", "y", "x"],
                                      coords={"return_period": target_return_periods,
                                              "y": ds_zsmax.y,
                                              "x": ds_zsmax.x,
                                              "spatial_ref":ds_zsmax['spatial_ref']})
return_periods_data_xr.to_netcdf('return_period_waterlevels.nc')

In [64]:
import hydromt
from scipy import ndimage
yml_base = r'Z:\Data-Expansion\users\lelise\data\data_catalog_BASE_Carolinas.yml'
cat = hydromt.DataCatalog(yml_base)
dep = cat.get_rasterdataset(r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter2_PGW\sfincs\subgrid\dep_subgrid_20m.tif')
target_shape = dep.shape

In [70]:
x_match = rda.coords['x'].equals(dep.coords['x'])
y_match = rda.coords['y'].equals(dep.coords['y'])
print(f"x dimensions match: {x_match}")
print(f"y dimensions match: {y_match}")

# Optionally, visualize a specific return period (e.g., 100-year return period)
for select in target_return_periods:
    if select == 2:
        continue
    rpd = return_periods_data_xr.sel(return_period=select)

    scaling_factors = [target_shape[i] / rpd.shape[i] for i in range(len(rpd.shape))]
    ra = ndimage.zoom(input=rpd, zoom=scaling_factors, order=1, output='float32', mode='grid-constant', cval=np.nan, prefilter=False, grid_mode=True)
    rda = xr.DataArray(ra,
                       dims=rpd.dims,
                       coords={dim: np.linspace(rpd.coords[dim].min(), rpd.coords[dim].max(),
                                                target_shape[i]) for i, dim in enumerate(rpd.dims)},
                       attrs=da.attrs)
    rda['spatial_ref'] = rpd['spatial_ref']

    #rda_regridded = rpd.interp(x=dep.coords['x'], y=dep.coords['y'], method='linear')
    rda.raster.to_raster(f'{select}yr_WSE.tif', nodata=np.nan)

    plt.imshow(rda_regridded, cmap='coolwarm', origin='lower')
    plt.colorbar(label='Water Level')
    plt.title(f'{select}-year Return Period Water Levels')
    plt.savefig(f'{select}-year Return Period Water Levels')
    plt.close()

    hmax = (rda_regridded - dep).compute()
    hmax['spatial_ref'] = rda['spatial_ref']
    hmax.raster.to_raster(f'{select}yr_hmax_20msbg.tif', nodata=np.nan)


x dimensions match: False
y dimensions match: False
